In [1]:
# Installing dependencies
#!pip install praw
#!pip install psaw
#!pip install yfinance

In [2]:
import praw
from psaw import PushshiftAPI
import json
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# Suppress warning messages
import warnings
#warnings.filterwarnings('ignore')

In [3]:
# Load client_id, secret_id, and user_agent
with open('info.json') as f:
     info = json.load(f)
        
info = dict(info)

In [4]:
# Initialize Reddit and PushshiftAPI instances
reddit = praw.Reddit(client_id=info["client_id"], user_agent=info["user_agent"], client_secret=info["client_secret"])
api = PushshiftAPI(reddit)

In [5]:
# Store results of a search in a DataFrame
subm_dicts = [{k:getattr(praw_obj, k) for k in vars(praw_obj)} for praw_obj in api.search_submissions(subreddit='stocks', q="TWTR", filter=['url','author', 'title', 'subreddit'], limit=100)]
df = pd.DataFrame(subm_dicts)
df

C:\Users\zdude\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


,comment_limit,comment_sort,_reddit,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,...,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,_fetched,_comments_by_id,link_flair_template_id
0,2048,confidence,<praw.reddit.Reddit object at 0x00000231DB56E070>,None,stocks,#Good morning traders and investors of the r/s...,t2_eaak0,False,None,0,...,False,https://www.reddit.com/r/stocks/comments/txlkg...,3856752,1.649250e+09,0,None,False,False,{},NaN
1,2048,confidence,<praw.reddit.Reddit object at 0x00000231DB56E070>,None,stocks,[removed],t2_4yrm7th0,False,None,0,...,False,https://www.reddit.com/r/stocks/comments/txef6...,3856752,1.649221e+09,0,None,False,False,{},866eb162-65e6-11e5-a903-1252b640afe9
2,2048,confidence,<praw.reddit.Reddit object at 0x00000231DB56E070>,None,stocks,[removed],t2_3u5upg0j,False,None,0,...,False,https://www.reddit.com/r/stocks/comments/twykl...,3856752,1.649175e+09,0,None,False,False,{},5a4c814a-65e6-11e5-b65e-122ab0778f8b
3,2048,confidence,<praw.reddit.Reddit object at 0x00000231DB56E070>,None,stocks,#Good morning traders and investors of the r/s...,t2_eaak0,False,None,0,...,False,https://www.reddit.com/r/stocks/comments/twts1...,3856752,1.649162e+09,0,None,False,False,{},NaN
4,2048,confidence,<praw.reddit.Reddit object at 0x00000231DB56E070>,None,stocks,[removed],t2_153n7s,False,None,0,...,False,https://www.reddit.com/r/stocks/comments/tw47e...,3856752,1.649085e+09,0,None,False,False,{},5a4c814a-65e6-11e5-b65e-122ab0778f8b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2048,confidence,<praw.reddit.Reddit object at 0x00000231DB56E070>,None,stocks,[removed],t2_2me1cihr,False,None,0,...,False,https://www.reddit.com/r/stocks/comments/n1w56...,3856752,1.619796e+09,0,None,False,False,{},NaN
96,2048,confidence,<praw.reddit.Reddit object at 0x00000231DB56E070>,None,stocks,[removed],t2_46aro24r,False,None,0,...,False,https://www.reddit.com/r/stocks/comments/n0tdi...,3856752,1.619658e+09,0,None,False,False,{},NaN
97,2048,confidence,<praw.reddit.Reddit object at 0x00000231DB56E070>,None,stocks,"**PsychoMarket Recap - Monday, April 26, 2021*...",t2_7gtjd4c0,False,None,0,...,False,https://www.reddit.com/r/stocks/comments/mz8dh...,3856752,1.619472e+09,1,None,False,False,{},NaN
98,2048,confidence,<praw.reddit.Reddit object at 0x00000231DB56E070>,None,stocks,"**PsychoMarket Recap - Thursday, April 22, 202...",t2_7gtjd4c0,False,None,0,...,False,https://www.reddit.com/r/stocks/comments/mwegi...,3856752,1.619125e+09,1,None,False,False,{},NaN


In [6]:
# Get historical stock data for a ticker
twtr = yf.download('TWTR', progress=True)
twtr

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-11-07,45.099998,50.090000,44.000000,44.900002,44.900002,117701600
2013-11-08,45.930000,46.939999,40.689999,41.650002,41.650002,27925300
2013-11-11,40.500000,43.000000,39.400002,42.900002,42.900002,16113900
2013-11-12,43.660000,43.779999,41.830002,41.900002,41.900002,6316700
2013-11-13,41.029999,42.869999,40.759998,42.599998,42.599998,8688300
...,...,...,...,...,...,...
2022-03-31,39.110001,39.230000,38.410000,38.689999,38.689999,13208300
2022-04-01,39.160000,39.849998,39.000000,39.310001,39.310001,12122600
2022-04-04,47.869999,51.369999,46.860001,49.970001,49.970001,268465400


## Processing Tickers

In [7]:
# At close, calculate the real and percent change since last close
def get_diff(ticker_data):
    df = ticker_data.copy()
    real = []
    percent = []
    for index, row in df.reset_index().iterrows():
        if(index == 0):
            real.append(0)
            percent.append(0)
        else:
            real.append(row["Close"]-df.iloc[index-1]["Close"])
            percent.append(real[-1]/df.iloc[index-1]["Close"])
        print(index)
    return real, percent

In [99]:
# Get the reddit posts that mention a certain ticker n days before a large change in stock price
def get_pre_change_posts(ticker, ticker_gain, days=1, limit=100, subreddit="stocks"):
    df = None
    for index, row in ticker_gain.iterrows():
        start_date = datetime.fromtimestamp(row["Date"].timestamp()) + timedelta(hours=6, days=-days)
        end_date = datetime.fromtimestamp(row["Date"].timestamp()) + timedelta(hours=6)
        
        # TODO: Check whether comments would be better than submissions
        
        submissions = api.search_comments(after=start_date, before=end_date, q=ticker, subreddit=subreddit, filter=['url','author', 'title', 'subreddit'], limit=limit)
        if(df is None):
            df = pd.DataFrame([{k:getattr(praw_obj, k) for k in vars(praw_obj)} for praw_obj in submissions])
        else:
            df = df.append([{k:getattr(praw_obj, k) for k in vars(praw_obj)} for praw_obj in submissions], ignore_index=True)
    return df

In [100]:
def word_counts(df, column="body", min_letters=3):
    counts = {}
    for i in list(df[column]):
        for j in i.split(" "):
            j = ''.join(k for k in j if k.isalnum())
            # Exclude words that are likely tickers
            if(j == j.upper() and len(j) > 1 and len(j) <= 5):
                pass
            elif(len(j) < 3):
                pass
            elif(j not in counts.keys()):
                counts[j.lower()] = 1
            else:
                counts.update({j.lower():counts.get(j.lower())+1})
    return counts

In [101]:
def remove_shared_keys(dict_a, dict_b):
    a = dict_a.copy()
    b = dict_b.copy()
    
    to_remove = []
    for i in a:
        if(i in b):
            to_remove.append(i)
    for i in to_remove:
        a.pop(i)
        b.pop(i)
    return a, b

In [102]:
def remove_infrequent_words(dict_a, min_count=2):
    d = dict_a.copy()
    
    to_remove = []
    for i, x in d.items():
        if(x < min_count):
            to_remove.append(i)
            
    for i in to_remove:
        d.pop(i)
        
    return d

In [110]:
# Generate information for a given ticker
def process_ticker(ticker, gain_cutoff=0.05, loss_cutoff=0.05, limit=100, days=1):
    ticker_data = yf.download(ticker, progress=False)
    ticker_data.reset_index(inplace=True)
    real, percent = get_diff(ticker_data)
    
    ticker_data["Real_Change"] = real
    ticker_data["Percent_Change"] = percent
    
    ticker_gain = ticker_data[ticker_data["Percent_Change"] > gain_cutoff]
    ticker_loss = ticker_data[ticker_data["Percent_Change"] < -loss_cutoff]
    
    pre_gain = get_pre_change_posts(ticker, ticker_gain, days, limit)
    pre_loss = get_pre_change_posts(ticker, ticker_loss, days, limit)
    
    gain_wc = dict(sorted(word_counts(pre_gain).items(), key=lambda x: x[1], reverse=True))
    loss_wc = dict(sorted(word_counts(pre_loss).items(), key=lambda x: x[1], reverse=True))
    
    gain_freq = remove_infrequent_words(gain_wc)
    loss_freq = remove_infrequent_words(loss_wc)
    
    gain_only, loss_only = remove_shared_keys(gain_freq, loss_freq)
    
    return gain_only, loss_only

In [112]:
#gain_only, loss_only = process_ticker("FB")

C:\Users\zdude\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\zdude\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\zdude\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [113]:
#gain_only

{'down': 30,
 'same': 22,
 'here': 19,
 'been': 17,
 'buying': 16,
 'does': 14,
 'did': 9,
 'three': 9,
 'then': 9,
 'quite': 8,
 'pandemic': 8,
 'meme': 8,
 'popular': 7,
 'love': 7,
 'income': 7,
 'teens': 7,
 'best': 6,
 'stop': 6,
 'came': 6,
 'ahead': 6,
 'aapl': 6,
 'smaller': 6,
 'unless': 6,
 'fuck': 6,
 'expensive': 6,
 'banned': 6,
 'well': 5,
 'outside': 5,
 'matters': 5,
 'peoples': 5,
 'begin': 5,
 'keeps': 5,
 'consider': 5,
 'continues': 5,
 'shopping': 5,
 'tok': 5,
 'decade': 5,
 'cannot': 5,
 'presence': 4,
 'starting': 4,
 'youre': 4,
 'question': 4,
 'hear': 4,
 'support': 4,
 'versus': 4,
 'niche': 4,
 'permanent': 4,
 'situation': 4,
 'non': 4,
 'posted': 4,
 'shift': 4,
 'addicted': 4,
 'exist': 4,
 'king': 4,
 'exactly': 4,
 'asking': 4,
 'plenty': 4,
 'evil': 4,
 'quick': 4,
 'seems': 4,
 'build': 4,
 'interested': 4,
 'pre': 4,
 'longterm': 4,
 'lucky': 4,
 'fair': 4,
 'learn': 4,
 'mainly': 4,
 'elsewhere': 4,
 'green': 4,
 'left': 4,
 'digital': 4,
 'feels':

In [114]:
#loss_only

{'still': 33,
 'too': 29,
 'sell': 21,
 'why': 16,
 'put': 16,
 'bad': 14,
 'all': 13,
 'hold': 12,
 'since': 12,
 'flows': 12,
 'tell': 9,
 'advantage': 8,
 'analysis': 8,
 'search': 8,
 'realize': 8,
 'luck': 8,
 'happens': 8,
 'third': 8,
 'kids': 8,
 'drops': 8,
 'meta': 8,
 'earning': 7,
 'interest': 7,
 'possible': 7,
 'overall': 7,
 'case': 7,
 'access': 7,
 'line': 7,
 'markets': 7,
 'small': 7,
 'makes': 6,
 'usually': 6,
 'average': 6,
 'man': 6,
 'willing': 6,
 'owned': 6,
 'bottom': 6,
 'info': 6,
 'taking': 6,
 'offer': 6,
 'current': 6,
 'his': 6,
 'forecast': 6,
 'buybacks': 6,
 'early': 5,
 'help': 5,
 'lost': 5,
 'opinion': 5,
 'volatility': 5,
 'costs': 5,
 'evidence': 5,
 'valuable': 5,
 'gross': 5,
 'normal': 5,
 'percent': 5,
 'family': 5,
 'party': 5,
 'couldnt': 5,
 'werent': 5,
 'private': 5,
 'life': 5,
 'cruise': 5,
 'analysts': 5,
 'ecom': 5,
 'monetized': 5,
 'investor': 4,
 'yourself': 4,
 'public': 4,
 'chance': 4,
 'sort': 4,
 'emotion': 4,
 'emotional': 

In [ ]:
all_gain = []
all_loss = []
for ticker in ["TWTR", "FB", "MSFT", "ADBE", "AAPL", "SNAP", "AMZN", "NCL", "DIS", "NFLX"]:
    gain_only, loss_only = process_ticker(ticker)
    all_gain.append(gain_only)
    all_loss.append(loss_only)
    print(ticker)

C:\Users\zdude\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\zdude\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\zdude\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


TWTR


C:\Users\zdude\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\zdude\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\zdude\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


FB


C:\Users\zdude\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\zdude\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\zdude\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


MSFT


C:\Users\zdude\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\zdude\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\zdude\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [ ]:
def combine_dict_list(list_of_dicts):
    single_dict = {}
    for d in list_of_dicts:
        for i in d:
            if(i not in single_dict):
                single_dict[i] = d.get(i)
            else:
                single_dict.update({i:single_dict.get(i)+d.get(i)})
    return single_dict

In [ ]:
gain, loss = (combine_dict_list(all_gain), combine_dict_list(all_loss))

In [ ]:
gain

In [ ]:
loss